### Char-RNN Trump-like text generation - TESTING

In [5]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


gpg: keybox '/tmp/tmpg9zge8nf/pubring.gpg' created
gpg: /tmp/tmpg9zge8nf/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [1]:
import sys
import numpy as np
import string

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation

LAYER_COUNT = 4
HIDDEN_LAYERS_DIM = 512

Using TensorFlow backend.


In [5]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [6]:
cd drive

/content/drive


In [7]:
cd char-rnn-master/

/content/drive/char-rnn-master


In [9]:
cd we

[Errno 2] No such file or directory: 'we'
/content/drive/char-rnn-master


In [2]:
# generic vocabulary
characters = list(string.printable)
characters.remove('\x0b')
characters.remove('\x0c')

VOCABULARY_SIZE = len(characters)
characters_to_ix = {c:i for i,c in enumerate(characters)}
print("vocabulary len = %d" % VOCABULARY_SIZE)
print(characters)

vocabulary len = 98
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' ', '\t', '\n', '\r']


In [0]:
test_model = Sequential()
for i in range(LAYER_COUNT):
    test_model.add(
            LSTM(
                HIDDEN_LAYERS_DIM, 
                return_sequences=True if (i!=(LAYER_COUNT-1)) else False,
                batch_input_shape=(1, 1, VOCABULARY_SIZE),
                stateful=True
            )
        )
test_model.add(Dense(VOCABULARY_SIZE))
test_model.add(Activation('softmax'))
test_model.compile(loss='categorical_crossentropy', optimizer="adam")

In [0]:
test_model.load_weights(
    "./weights/weights_4-512-lstm_loss1.2550_val-loss1.2443"
)

In [0]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array"""
    # from fchollet/keras
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def predict_next_char(model, current_char, diversity=1.0):
    """Predict the next character from the current one"""
    x = np.zeros((1, 1, VOCABULARY_SIZE))
    x[:,:,characters_to_ix[current_char]] = 1
    y = model.predict(x, batch_size=1)
    next_char_ix = sample(y[0,:], temperature=diversity)
    next_char = characters[next_char_ix]
    return next_char

def generate_text(model, seed="I am", count=140):
    """Generate characters from a given seed"""
    model.reset_states()
    for s in seed[:-1]:
        next_char = predict_next_char(model, s)
    current_char = seed[-1]

    sys.stdout.write("["+seed+"]")
    
    for i in range(count - len(seed)):
        next_char = predict_next_char(model, current_char, diversity=0.5)
        current_char = next_char
        sys.stdout.write(next_char)
    print("...\n")

In [14]:
for i in range(5):
    generate_text(
        test_model,
        seed="Real "
    )

[Real ]Congress in Virginia and Golf Course http://t.co/1zyukiZY @BarackObama h

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


as not a complete and price is a great honor to be a healthy da...

[Real ]@TrumpToronto - @Joan_Rivers is a lot of important calls for everyone for his deal is a total performance. @MissUniverse Happy Birthday...

[Real ]http://t.co/KS15q94I @BarackObama will be having a few base and the way. We are sure the future @MittRomney is a travel thing http://t....

[Real ]https://t.co/ySqq8yOy4L "@TheNewsJustin: @realDonaldTrump I don't think you are a foreign president and a speech today!" Thanks. "@Jaso...

[Real ]@realDonaldTrump Mr Trump is such a great guy! Thanks. "@MarketArnosso: @realDonaldTrump @dannyzuker @BrandiGlanville I agree! What a g...

